Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [154]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass, field



## The *Nim* and *Nimply* classes

In [155]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [156]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
    
    def copy(self) -> 'Nim':
        new_nim = Nim(num_rows=len(self._rows), k=self._k)
        new_nim._rows = list(self._rows)
        return new_nim


## Sample (and silly) startegies 

In [157]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [158]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [159]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analyze(raw: Nim) -> dict:
    cooked = {"possible_moves": {}}
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analyze(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply




## (1,λ) ES


In [160]:
import random
from typing import Tuple

NUM_SETS = 5
LAMBDA = 20

@dataclass
class Individual:
    fitness: int
    genotype: Nim
    mutation_info: Tuple[int, int] = field(default_factory=lambda: (0, 0))

def fitness(state: Nim) -> int:
    num_non_empty_sets = sum(1 for x in state.rows if x > 0)

    if num_non_empty_sets == 1:
        return 100

    return nim_sum(state)

def mutate(ind: Individual) -> Individual:
    state = list(ind.genotype.rows)

    non_empty_sets = [i for i in range(NUM_SETS) if state[i] > 0]

    pos = random.choice(non_empty_sets)
    if state[pos] != 1:
        num_to_remove = random.randint(1, state[pos]-1)
    else:
        num_to_remove = 1

    
    if len(non_empty_sets) == 1:
        num_to_remove = state[pos] - 1 

    state[pos] -= num_to_remove

    new_state_nim = ind.genotype.copy()
    new_state_nim.nimming(Nimply(pos, num_to_remove))

    mutated_individual = Individual(
        fitness=0,
        genotype=new_state_nim,
        mutation_info=(pos, num_to_remove),
    )

    return mutated_individual



def adaptive(state: Nim,) -> Nimply:
    offspring = []

    for _ in range(LAMBDA):
        parent = Individual(
                    fitness=0,
                    genotype=state
                )
        child = mutate(parent)
        offspring.append(child)

    for individual in offspring:
        individual.fitness = fitness(individual.genotype)

    best_mutated_individual = max(offspring, key=lambda ind: ind.fitness)
   
    result_nimply = Nimply(
        row=best_mutated_individual.mutation_info[0],
        num_objects=best_mutated_individual.mutation_info[1],
    )
    return result_nimply

## Oversimplified match

In [161]:
from tqdm import tqdm

num_iteration = 1000
count_0 = 0
count_1 = 0

logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, adaptive)

for _ in tqdm(range(num_iteration)):
    nim = Nim(5)
    
    #logging.info(f"status: {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
        
    if player == 0:
        count_0 += 1
    else:
        count_1 += 1

print(f"Number games won player_0: {count_0}, Percentage: {round(count_0 / num_iteration * 100)}%")
print(f"Number games won player_1: {count_1}, Percentage: {round(count_1 / num_iteration * 100)}%")


  1%|▏         | 13/1000 [00:00<00:16, 59.67it/s]

15
15
13
13
14
13
3
14
13
13
13
15
10
11
13
14
13
15
13
14
5
8
15
13
15
15
5
15
15
13
6
13
9
15
4
15
9
15
15
9
1
8
8
10
10
8
8
6
10
10
10
7
8
8
10
8
1
10
8
10
8
8
8
8
8
5
8
8
8
8
3
7
8
8
6
8
8
8
8
6
5
3
2
1
1
2
5
5
5
5
2
5
5
5
1
1
2
5
5
1
0
0
100
100
0
100
100
100
0
0
100
100
100
100
0
0
100
100
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
12
11
13
6
11
14
11
1
12
11
11
7
11
8
8
11
14
12
12
6
14
13
14
14
14
12
14
11
4
13
14
14
5
12
14
14
12
5
14
11
6
6
1
4
6
6
6
6
6
6
0
1
6
6
6
6
6
1
3
6
100
100
100
100
100
100
2
2
100
2
100
0
100
100
3
3
100
100
3
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
10
12
12
12
12
15
12
6
15
15
2
9
12
11
0
3
12
12
10
15
7
4
7
7
4
3
3
4
4
4
4
4
2
4
7
7
4
4
4
4
4
4
4
4
4
4
4
1
4
4
4
4
4
1
4
4
4
4
4
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
5
0
6
6
1
6
5
6
6
2
1
1
5
6

  2%|▏         | 19/1000 [00:00<00:17, 54.79it/s]

2
2
2
1
2
2
1
4
4
2
1
2
0
1
7
2
1
1
2
7
7
7
7
4
7
7
7
4
4
7
7
7
7
4
4
7
4
7
7
7
2
7
3
7
7
1
4
7
7
7
4
3
1
7
4
4
4
3
4
4
7
7
6
6
6
4
6
3
3
7
6
6
6
2
4
6
6
4
4
6
0
5
5
5
0
3
5
3
0
1
1
5
3
5
1
5
1
3
3
5
3
100
0
100
3
3
3
100
0
100
100
100
3
3
100
100
100
100
0
3
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
0
7
0
0
0
7
0
7
0
6
0
3
3
0
3
3
3
0
3
3
7
7
7
2
5
7
0
7
7
5
7
7
5
5
4
7
7
4
7
4
7
3
4
4
6
4
4
0
4
4
6
4
4
4
7
7
6
4
4
6
0
2
0
0
0
0
2
2
2
2
0
0
0
0
0
0
2
2
2
2
0
0
0
0
0
100
0
0
100
100
0
0
0
0
0
100
100
0
100
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
15
10
13
13
15
12
15
15
15
12
13
2
15
2
6
13
15
12
15
4
2
13
15
1
13
11
11
13
4
15
7
7
12
13
0
0
12
2
13
10
0
10
8
8
8
8
8
10
10
6
6
8
8
9
8
8
8
1
1
8
0
2
0
0
0
2
0
0
0
0
0
0
0
0
2
0
0
2
0
0
2
2
2
0
0
2
2
0
0
2
0
0
0
2
0
2
2
2
2
2
0
0
100
100
0
0
0
0
0
100
0
0
0
100
0
0
100
0
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
1
3
1
4

  3%|▎         | 32/1000 [00:00<00:18, 53.46it/s]

...

5
2
5
6
5
0
3
1
2
1
5
2
2
7
6
0
2
2
0
5
4
4
3
4
4
2
4
4
4
3
0
4
3
4
4
4
4
3
4
4
6
6
0
7
7
4
4
4
4
7
7
6
7
0
4
0
0
0
0
7
5
0
5
5
0
1
1
5
1
1
0
0
5
0
3
0
1
5
1
0
100
3
3
0
100
0
0
0
100
100
100
100
3
100
100
100
0
100
3
3
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
14
1
11
14
14
14
14
13
14
12
14
14
3
13
14
11
13
14
12
13
14
13
9
14
14
13
5
13
0
14
14
3
1
13
12
12
12
10
14
12
8
15
15
15
15
2
13
2
15
15
6
2
15
15
6
3
0
15
15
15
11
11
9
9
9
9
11
11
4
9
9
9
11
11
9
9
2
11
4
9
9
9
9
0
9
9
9
9
9
3
9
9
9
9
9
9
9
7
5
3
2
2
2
2
2
1
2
2
2
1
2
2
2
2
2
2
2
2
2
2
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
8
13
15
15
15
12
15
15
11
6
2
3
12
15
15
15
12
15
8
15
12
1
12
12
12
14
0
1
12
8
14
12
14
14
11
14
14
14
12
1
9
5
9
7
9
9
9
9
9
2
9
9
9
9
9
6
9
9
9
9
4
4
4
4
1
4
4
4
4
4
4
4
4
1
4
4
2
4
4
4
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
6
1
3
3
2
2
4
2
3
3
4
0
3
7
7
4
2
4
1
7
7
7
3
0
4
7
7
4
3


 99%|█████████▉| 994/1000 [00:16<00:00, 42.32it/s]

0
6
0
6
3
6
3
0
0
3
6
6
3
0
3
1
1
1
1
0
0
0
0
1
0
3
0
0
3
0
0
0
2
2
0
2
2
0
2
0
2
2
2
3
2
2
2
0
2
2
2
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
11
13
12
0
11
14
2
8
12
11
15
11
11
11
1
7
8
2
8
11
15
8
13
13
15
15
13
15
15
2
15
15
15
15
15
2
15
5
9
15
10
10
13
13
13
4
10
15
13
3
13
13
8
6
2
6
13
13
15
5
11
4
9
11
1
10
9
12
12
6
9
6
4
9
11
10
1
4
11
6
2
0
100
0
0
100
100
3
100
100
3
100
100
100
100
3
2
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
14
14
9
6
2
9
2
11
3
6
10
10
12
9
9
9
9
10
9
0
14
13
13
14
15
14
13
13
13
14
15
15
13
14
13
13
13
13
13
13
2
5
1
1
6
2
5
2
4
5
2
7
2
2
7
1
5
4
5
4
5
5
5
5
2
5
5
5
5
3
5
5
5
5
5
5
5
5
5
3
0
0
100
0
0
100
100
100
0
100
100
0
100
100
100
100
0
100
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
14
14
14
14
14
14
14
14
14
14
13
14
13
14
13
3
14
11
14
14
0
7
10
10
7
8
10
4
8
8
5
10
10
8
8
10
8
8
10
8
8
1
6
7
7
8
4
8
1
3
8
6
7
5
4

100%|██████████| 1000/1000 [00:16<00:00, 58.86it/s]

7
5
5
6
6
5
7
6
5
6
5
6
5
5
5
6
3
6
5
6
2
2
100
100
100
100
3
3
5
5
100
0
100
2
100
5
100
100
100
2
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
14
11
8
13
13
13
4
3
13
15
15
10
14
1
15
15
13
14
2
15
15
15
15
15
10
8
7
7
15
15
15
15
15
15
0
15
15
15
0
15
8
8
4
8
8
3
11
8
11
11
11
5
8
8
8
4
0
11
3
6
7
7
0
2
0
0
7
1
1
2
7
6
0
7
7
1
1
7
7
7
0
100
3
0
3
100
3
100
0
100
0
100
100
3
3
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
9
14
15
6
5
11
1
9
9
12
9
10
9
15
9
1
9
11
13
12
5
5
6
5
6
5
4
5
6
4
6
6
6
5
4
3
4
2
4
4
7
3
7
3
7
5
7
7
7
3
5
0
3
6
7
6
7
3
7
0
1
1
7
1
7
1
2
5
7
7
5
5
7
5
7
5
2
7
7
0
100
0
0
0
100
0
0
0
0
100
0
0
0
100
100
0
0
100
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
4
10
9
9
12
9
9
5
9
0
15
11
9
12
12
9
0
11
14
9
6
8
7
8
10
7
5
10
10
10
8
0
8
10
10
10
8
10
8
8
8
8
2
10
8
8
7
11
5
10
8
11
1
11
8
11
8
11
4
5
9
9
9
9
9
9
9
9
9
9
9
7
9
9
9
9
9
9
9
3
7
7